In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from bias_transfer.bias_transfer.main import *

In [ ]:
force_cpu=False
lr=0.1
resume=False
cifar100=True

device = 'cuda' if torch.cuda.is_available() and not force_cpu else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

print('==> Load data..')
if cifar100:
    CIFAR100_TRAIN_MEAN = (0.5070751592371323, 0.48654887331495095, 0.4409178433670343)
    CIFAR100_TRAIN_STD = (0.2673342858792401, 0.2564384629170883, 0.27615047132568404)
    train_loader = get_training_data_loader(mean=CIFAR100_TRAIN_MEAN, std=CIFAR100_TRAIN_STD)
    test_loader = get_test_data_loader(mean=CIFAR100_TRAIN_MEAN, std=CIFAR100_TRAIN_STD)
else:  # CIFAR10
    CIFAR10_TRAIN_MEAN = (0.4914, 0.4822, 0.4465)
    CIFAR10_TRAIN_STD =(0.2023, 0.1994, 0.2010)
    train_loader = get_training_data_loader(mean=CIFAR10_TRAIN_MEAN, std=CIFAR10_TRAIN_STD, cifar100=False)
    test_loader = get_test_data_loader(mean=CIFAR10_TRAIN_MEAN, std=CIFAR10_TRAIN_STD, cifar100=False)



In [ ]:
# Model
print('==> Building model..')
net = resnet50(num_classes=100 if cifar100 else 10)
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

if resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/ckpt.pth')
    net.load_state_dict(checkpoint['net'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
# train_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=settings.MILESTONES, gamma=0.2)  # learning rate decay
# iter_per_epoch = len(cifar100_training_loader)
# warmup_scheduler = WarmUpLR(optimizer, iter_per_epoch * args.warm)

for epoch in range(start_epoch, start_epoch + 200):
    train(net, train_loader, device, epoch, optimizer, criterion)
    test(net, test_loader, device, epoch, criterion, best_acc)
